# Final Project - Presentation Version

## Group Name: Lambda

### Student Names
1. Jian Wang
2. Chong Geng
3. Alan Perry
4. Divya Bhargavi
5. Robert Sandor

## Feature Engineering

One of the major tasks of this project was to generate numerical features based upon the text we were provided. After doing some research, we decided upon a number of features that we thought might work.

In [1]:
from Feature_Engineering.feature_engineering import *

/Users/robertsandor/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [2]:
products = pd.read_csv('Data/product_descriptions.csv')
train = pd.read_csv('Data/train.csv', encoding='ISO-8859-1')
attributes = pd.read_csv('Data/attributes.csv', encoding='ISO-8859-1')

In [3]:
# BEWARE: this takes ~2.5 min to run
attrib_per_product = attrib_stack(attributes)

In [4]:
train, attrib_per_product = join_attrib(train, attrib_per_product)
train = search_term_in_attrib(train)

In [5]:
# BEWARE: this takes ~4 min to run
train = color_df(attributes, train)
train = search_title_lev_dist(train)

In [6]:
train = search_brand_lev_dist(train, attributes)

In [7]:
train_temp = train.drop(['id', 'name', 'value', 'search_term_split',
                         'color', 'product_title_clean', 'brand'], axis=1)

In [8]:
glove_file = 'Data/glove.6B.300d.txt'
glove_dic = make_dictionary(glove_file)

In [9]:
modified_train = train.set_index('product_uid').join(
        products.set_index('product_uid'))
modified_train = modified_train.reset_index()

In [19]:
# BEWARE: this takes ~26 min to run
# modified_train = feature_engineering(train, products, glove_dic)

In [10]:
modified_train = createCleanedTermsCol(modified_train)

In [14]:
# BEWARE: this takes ~3.75 min to run
modified_train = createStemmedCols(modified_train)

In [15]:
modified_train = createLemmatizedCols(modified_train)

In [11]:
# BEWARE: this takes 12.5 min to run
modified_train = findNeighborsInCorpus(modified_train, glove_dic, glove_file)

In [12]:
modified_train = createLengthCols(modified_train)

In [13]:
modified_train = findCleanedTermsInCorpusCols(modified_train)

In [16]:
modified_train = findStemmedTermsInCorpusCols(modified_train)

In [17]:
modified_train = findLemmatizedTermsInCorpusCols(modified_train)

In [18]:
modified_train = createEntropyCols(modified_train)

In [19]:
# BEWARE: this takes 10.5 min to run
modified_train = createLCSCols(modified_train)

In [20]:
modified_train = createJaccardIndexCols(modified_train)

In [21]:
# BEWARE: this takes ~4 min to run
createJaroCol(modified_train, "search_term",
              "product_description", "jscore_query_desc")
createJaroCol(modified_train, "search_term",
              "product_title", "jscore_query_title")

In [21]:
# BEWARE: this takes ~8.5 min to run
modified_train = createSWscoreCol(
    modified_train, "search_term", "product_title", "search_title_SW")

In [23]:
# BEWARE: this takes ~1hr 46m to run
modified_train = createSWscoreCol(
    modified_train, "search_term", "product_description", "search_desc_SW")

In [25]:
# BEWARE: this takes ~14hr 35m to run
modified_train = createNCDCol(
    modified_train, "search_term", "product_title", "NCD_query_title")

In [112]:
modified_train = add_prod_description_column(modified_train)

In [114]:
modified_train = add_word_count_features(modified_train)

In [119]:
# BEWARE: this takes ~3m to run
modified_train = add_num_attrib_per_prod_column(modified_train, attributes)

In [120]:
# BEWARE, this takes ~5h 12m to run
modified_train = find_n_tfidf_highest_scores(modified_train, 5)

In [121]:
modified_train = find_tfidf_words_in_search(modified_train)

In [122]:
modified_train = add_tfidf_col(modified_train)
modified_train['num_attrib'].fillna(0, inplace=True)

In [156]:
print(modified_train.columns.values)
modified_train[:3]

['product_uid' 'id' 'product_title' 'search_term' 'relevance' 'name'
 'value' 'search_term_split' 'search_term_in_attrib' 'color'
 'color_in_search_term' 'product_title_clean' 'min_levenstein_dist_title'
 'brand' 'min_levenstein_dist_brand' 'product_description' 'cleaned_terms'
 'stemmed_terms' 'stemmed_title' 'stemmed_desc' 'lemmatized_terms'
 'lemmatized_title' 'lemmatized_desc' 'terms_neighbour' 'clean_length'
 'title_length' 'desc_length' 'clean_terms_in_title' 'clean_terms_in_desc'
 'stemmed_terms_in_title' 'stemmed_terms_in_desc'
 'lemmatized_terms_in_title' 'lemmatized_terms_in_desc'
 'neighbours_in_title' 'neighbours_in_desc' 'search_terms_entropy'
 'title_entropy' 'jaccard_index_title' 'jaccard_index_desc' 'lcs_title'
 'lcs_desc' 'jscore_query_desc' 'jscore_query_title' 'search_title_SW'
 'search_desc_SW' 'NCD_query_title' 'total_description'
 'num_words_in_description' 'num_stop_words' 'num_search_words'
 'num_attrib' 'tfidf' 'tfidf_search_common']


,product_uid,id,product_title,search_term,relevance,name,value,search_term_split,search_term_in_attrib,color,...,search_title_SW,search_desc_SW,NCD_query_title,total_description,num_words_in_description,num_stop_words,num_search_words,num_attrib,tfidf,tfidf_search_common
0,100001,2,Simpson Strong-Tie 12-Gauge Angle,angle bracket,3.0,"['Bullet01', 'Bullet02', 'Bullet03', 'Bullet04...","{'stronger', 'consist', 'extra', 'instal', 'jo...","[angl, bracket]",1,{''},...,1.0,4.0,0.107077,Simpson Strong-Tie 12-Gauge AngleNot only do a...,79,0,2,15.0,"[angl, simpson, strong, tie, project]",0
1,100001,3,Simpson Strong-Tie 12-Gauge Angle,l bracket,2.5,"['Bullet01', 'Bullet02', 'Bullet03', 'Bullet04...","{'stronger', 'consist', 'extra', 'instal', 'jo...",[bracket],0,{''},...,0.0,0.0,0.107077,Simpson Strong-Tie 12-Gauge AngleNot only do a...,79,0,2,15.0,"[angl, simpson, strong, tie, project]",0
2,100002,9,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,deck over,3.0,"['Application Method', 'Assembled Depth (in.)'...","{'represent', 'durabl', 'behr', 'slip', 'follo...",[deck],1,"{'Tans', 'Browns', 'Tugboat'}",...,0.0,3.0,0.109091,BEHR Premium Textured DeckOver 1-gal. #SC-141 ...,109,1,2,35.0,"[concret, deckov, behr, textur, deck]",0


## Load Data

Becaues of the time-consuming nature of the feature engineering, we saved the data into a csv. If you don't want to run all of the previous functions (which takes over 16 hours to run), just start with the cells below.

In [50]:
zip_data = zipfile.ZipFile('Data/final_dataset.csv.zip')
zip_data.extractall()
modified_train = pd.read_csv('Data/final_dataset.csv')
modified_train = modified_train.drop('Unnamed: 0', axis=1)
assert(len(modified_train.columns) == 49)

In [51]:
X_train = modified_train[['clean_length', 'title_length',
                          'desc_length', 'clean_terms_in_title',
                          'clean_terms_in_desc', 
                          'min_levenstein_dist_title', 'min_levenstein_dist_brand',
                          'stemmed_terms_in_title', 'stemmed_terms_in_desc',
                          'lemmatized_terms_in_title', 'lemmatized_terms_in_desc',
                          'neighbours_in_title', 'neighbours_in_desc', 'search_terms_entropy',
                          'title_entropy', 'jaccard_index_title', 'jaccard_index_desc', 'lcs_title',
                          'lcs_desc', 'jscore_query_desc', 'jscore_query_title', 'search_title_SW',
                          'search_desc_SW', 'NCD_query_title', 'num_words_in_description', 'num_stop_words',
                          'num_search_words', 'tfidf_search_common', 'num_attrib']]
y_train = modified_train[['relevance']]

Since we can't see the relevancy scores of the test set, we decided to split the training set further into our own training and test set.

In [52]:
train_data, test_data, train_target, test_target = train_test_split(X_train,
                                                                    y_train)

In [53]:
assert(int(math.floor(len(X_train) * .75)) == len(train_data))
assert(len(train_data) == len(train_target))
assert(int(math.ceil(len(X_train) * .25)) == len(test_data))
assert(len(test_data) == len(test_target))

## Baseline

As a simple baseline, we considered the RMSE of a completely random model.

In [39]:
print(len(y_train)) # no empty relevancy score.

74071


Here we perform a randomization test on the relevancy scores where we permute the relevancy scores. In that way, we can keep the same distribution and effectively get a completely random model

In [62]:
random_score = np.random.permutation(y_train)
random_chance_performance = math.sqrt(mean_squared_error(random_score, y_train))
print(random_chance_performance)

0.7559194204331985


# Model Fitting

## Linear Models

For our simplest model, we decided to use linear regression.

In [7]:
lin_reg_model = LinearRegression()
lin_reg_model.fit(train_data, train_target)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [8]:
predicted = lin_reg_model.predict(test_data)
print(predicted[:5])
print(test_target[:5])

[[2.222388  ]
 [2.05996623]
 [2.29824388]
 [2.32938087]
 [2.42778731]]
       relevance
6436        1.67
52863       1.67
63768       1.67
50948       2.67
30097       3.00


After some exploration, we realized that some of the predictions produced by linear regression exceeded the bounds of the relevancy score metric. To account for that, we performed a min-max scaling to get the predictions within the bounds.

In [9]:
scaler = MinMaxScaler(feature_range=(1.0, 3.0))
scaled_linear_predicted = scaler.fit_transform(predicted)
print(scaled_linear_predicted[:5])
print(test_target[:5])

[[2.15978968]
 [2.01318068]
 [2.22826052]
 [2.25636613]
 [2.3451921 ]]
       relevance
6436        1.67
52863       1.67
63768       1.67
50948       2.67
30097       3.00


For good measure, we considered alternative linear models that have regularization like Lasso, Ridge and ElasticNet to evaluate if there was overfitting. 

In [65]:
def grid_search_models_rmse(models, model_params, train_data, train_target):
    """
    :param models:          a list of pipelines of models
    :param model_params:    a list of dictionaries with the model parameters
    """
    print(len(train_data))
    print(train_data[:3])
    print(train_data.iloc[:3, 0])
    best_models = []
    for model in zip(models, model_params):
        gs = GridSearchCV(estimator=model[0],
                          param_grid=model[1],
                          scoring='neg_mean_squared_error',
                          cv=5)
        if type(train_data) != np.ndarray:
            y = train_data.values.ravel()
            train_data = np.array(y).astype(float)
        gs.fit(train_data, train_target)
        best_models_alt_linear.append(
            (sqrt(-1 * gs.best_score_), gs.best_params_, model[0]))
    return best_models

In [144]:
# BEWARE: this takes ~11m to run
alt_linear_models = [Lasso(),
                     Ridge(),
                     ElasticNet()]

alt_linear_grid_params = [{'alpha': np.linspace(start=0.25, stop=1.0, num=4),
                           'normalize': [False, True],
                           'selection': ['cyclic', 'random']},
                          {'alpha': np.linspace(start=0.5, stop=2.0, num=4),
                           'normalize': [False, True],
                           'solver': ['svd', 'lsqr', 'sag', 'saga']},
                          {'alpha': np.linspace(start=0.5, stop=2.0, num=4),
                           'l1_ratio': np.linspace(start=0.25, stop=1.0, num=4),
                           'normalize': [False, True],
                           'selection': ['cyclic', 'random']}]


best_models_alt_linear = []
for model in zip(alt_linear_models, alt_linear_grid_params):
    gs = GridSearchCV(estimator=model[0],
                      param_grid=model[1],
                      scoring='neg_mean_squared_error',
                      cv=5)
    if type(y_train) != np.ndarray:
        y = y_train.values.ravel()
        y_train = np.array(y).astype(float)
    gs.fit(X_train, y_train.ravel())
    best_models_alt_linear.append(
        (sqrt(-1 * gs.best_score_), gs.best_params_, model[0]))

print(best_models_alt_linear)

[(0.5308711292115721, {'alpha': 0.25, 'normalize': False, 'selection': 'random'}, Lasso(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)), (0.4946356430077207, {'alpha': 0.5, 'normalize': False, 'solver': 'svd'}, Ridge(alpha=1.0, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)), (0.5186387637495514, {'alpha': 0.5, 'l1_ratio': 0.25, 'normalize': False, 'selection': 'cyclic'}, ElasticNet(alpha=1.0, copy_X=True, fit_intercept=True, l1_ratio=0.5,
      max_iter=1000, normalize=False, positive=False, precompute=False,
      random_state=None, selection='cyclic', tol=0.0001, warm_start=False))]


In [145]:
best_model_alt_linear = sorted(
    best_models_alt_linear, key=lambda model: model[0])[0]
best_model_alt_linear = best_model_alt_linear[2].__class__(
    **best_model_alt_linear[1])
best_model_alt_linear.fit(train_data, train_target)
alt_linear_predicted = best_model_alt_linear.predict(test_data)
print(alt_linear_predicted[:5])
print(test_target[:5])

[[2.10183055]
 [2.18559358]
 [2.12364928]
 [2.50951735]
 [2.57230042]]
       relevance
49710       2.67
58951       1.33
45702       2.33
63338       2.00
71218       2.00


## Tree-based Models

After considering various linear models, we decided to test if tree-based models could improve upon the performance of the linear models.

In [146]:
# BEWARE: this takes ~26m to run
tree_models = [RandomForestRegressor(),
               AdaBoostRegressor()]

grid_params_tree = [{'n_estimators': range(1, 30, 5),
                     'max_features': ['auto', 'sqrt', 'log2', None]},
                    {'n_estimators': range(1, 30, 5),
                     'loss': ['linear', 'square'],
                     'learning_rate': np.linspace(start=0.5, stop=1.5, num=5)}]
best_models_tree = []
for model in zip(tree_models, grid_params_tree):
    gs = GridSearchCV(estimator=model[0],
                      param_grid=model[1],
                      scoring='neg_mean_squared_error',
                      cv=5)
    if type(y_train) != np.ndarray:
        y = y_train.values.ravel()
        y_train = np.array(y).astype(float)
    gs.fit(X_train, y_train.ravel())
    best_models_tree.append(
        (sqrt(-1 * gs.best_score_), gs.best_params_, model[0]))

print(best_models_tree)

[(0.4766320708963225, {'max_features': 'log2', 'n_estimators': 26}, RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)), (0.5021121390851352, {'learning_rate': 0.75, 'loss': 'linear', 'n_estimators': 6}, AdaBoostRegressor(base_estimator=None, learning_rate=1.0, loss='linear',
         n_estimators=50, random_state=None))]


In [147]:
# here we chose the best tree model
best_tree_model = sorted(best_models_tree, key=lambda model: model[0])[0]
best_tree_model = best_tree_model[2].__class__(**best_tree_model[1])
best_tree_model.fit(train_data, train_target)
tree_predicted = best_tree_model.predict(test_data)
print(tree_predicted[:5])
print(test_target[:5])

/Users/robertsandor/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


[2.22153846 2.23038462 2.34615385 2.16653846 2.24076923]
       relevance
49710       2.67
58951       1.33
45702       2.33
63338       2.00
71218       2.00


In [149]:
# BEWARE : this takes ~17m to run
rf_models = [Pipeline([('rf_reg', RandomForestRegressor())])]

grid_params_rf = [{'rf_reg__n_estimators': range(25, 151, 10),
                   'rf_reg__max_features': ['log2']}]
best_models_rf = []
for model in zip(rf_models, grid_params_rf):
    gs = GridSearchCV(estimator=model[0],
                      param_grid=model[1],
                      scoring='neg_mean_squared_error',
                      cv=5)
    if type(y_train) != np.ndarray:
        y = y_train.values.ravel()
        y_train = np.array(y).astype(float)
    gs.fit(X_train, y_train.ravel())
    best_models_rf.append(
        (sqrt(-1 * gs.best_score_), gs.best_params_, model[0]))

print(best_models_rf)

[(0.46981443222286356, {'rf_reg__max_features': 'log2', 'rf_reg__n_estimators': 135}, Pipeline(memory=None,
     steps=[('rf_reg', RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False))]))]


In [155]:
best_rf_model = sorted(best_models_rf, key=lambda model: model[0])[0]
best_rf_model[1]['max_features'] = best_rf_model[1].pop('rf_reg__max_features')
best_rf_model[1]['n_estimators'] = best_rf_model[1].pop('rf_reg__n_estimators')
best_rf_model = (best_rf_model[0], best_rf_model[1], best_rf_model[2])

<class 'sklearn.ensemble.forest.RandomForestRegressor'>
{'rf_reg__max_features': 'log2', 'rf_reg__n_estimators': 135}
{'max_features': 'log2', 'n_estimators': 135}


In [156]:
best_rf_model = best_rf_model[2].steps[0][1].__class__(
    **best_rf_model[1])
best_rf_model.fit(train_data, train_target)
rf_predicted = best_rf_model.predict(test_data)
print(rf_predicted[:5])
print(test_target[:5])

/Users/robertsandor/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  after removing the cwd from sys.path.


[2.19407407 2.26325926 2.33103704 2.23274074 2.20696296]
       relevance
49710       2.67
58951       1.33
45702       2.33
63338       2.00
71218       2.00


To be thorough, we also considered a model that used boosting (Gradient Boosting) to see how boosting would compare to all of our models thus far.

In [12]:
# BEWARE: this takes ~14m to run
gb_model_pipeline = [Pipeline([('gb_reg', GradientBoostingRegressor())])]

grid_params_gb = [{'gb_reg__loss': ['ls', 'huber'],
                   'gb_reg__n_estimators': range(1, 26, 5),
                   'gb_reg__learning_rate': [0.1, 0.25, 0.5, 0.75, 0.9]}]
best_gb_models = []
for model in zip(gb_model_pipeline, grid_params_gb):
    gs = GridSearchCV(estimator=model[0],
                      param_grid=model[1],
                      scoring='neg_mean_squared_error',
                      cv=5)
    if type(y_train) != np.ndarray:
        y = y_train.values.ravel()
        y_train = np.array(y).astype(float)
    gs.fit(X_train, y_train.ravel())
    best_gb_models.append(
        (sqrt(-1 * gs.best_score_), gs.best_params_, model[0]))

print(best_gb_models)

[(0.48326051146343524, {'gb_reg__learning_rate': 0.5, 'gb_reg__loss': 'ls', 'gb_reg__n_estimators': 21}, Pipeline(memory=None,
     steps=[('gb_reg', GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False))]))]


In [13]:
best_gb_model = sorted(
    best_gb_models, key=lambda model: model[0])[0]
best_gb_model[1]['learning_rate'] = best_gb_model[1].pop(
    'gb_reg__learning_rate')
best_gb_model[1]['n_estimators'] = best_gb_model[1].pop(
    'gb_reg__n_estimators')
best_gb_model[1]['loss'] = best_gb_model[1].pop(
    'gb_reg__loss')

In [14]:
best_gb_model = best_gb_model[2].steps[0][1].__class__(
    **best_gb_model[1])

best_gb_model.fit(train_data, train_target)
gb_predicted = best_gb_model.predict(test_data)
print(gb_predicted[:5])
print(test_target[:5])

/Users/robertsandor/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


[2.2805891  2.00586902 2.33699338 2.18935758 2.3945759 ]
       relevance
6436        1.67
52863       1.67
63768       1.67
50948       2.67
30097       3.00


In [31]:
# BEWARE: this takes ~11m to run
refined_gb_model_pipeline = [Pipeline([('gb_reg', GradientBoostingRegressor())])]

grid_params_refined_gb = [{'gb_reg__loss': ['ls'],
                   'gb_reg__n_estimators': range(25, 151, 25),
                   'gb_reg__learning_rate': [0.4, 0.5, 0.6]}]
best_refined_gb_models = []
for model in zip(refined_gb_model_pipeline, grid_params_refined_gb):
    gs = GridSearchCV(estimator=model[0],
                      param_grid=model[1],
                      scoring='neg_mean_squared_error',
                      cv=5)
    if type(y_train) != np.ndarray:
        y = y_train.values.ravel()
        y_train = np.array(y).astype(float)
    gs.fit(X_train, y_train.ravel())
    best_refined_gb_models.append(
        (sqrt(-1 * gs.best_score_), gs.best_params_, model[0]))

print(best_refined_gb_models)

[(0.4788486584277938, {'gb_reg__learning_rate': 0.4, 'gb_reg__loss': 'ls', 'gb_reg__n_estimators': 150}, Pipeline(memory=None,
     steps=[('gb_reg', GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False))]))]


## Tree-Based Models - Reduced Dimensionality

After evaluating performance on Random Forest models, we wondered how applying PCA (Principal Component Analysis) to reduce the dimensionality of our data would affect performance.

In [162]:
# BEWARE: this takes ~6m to run
transformed_rf_models = [Pipeline([('scale', StandardScaler()),
                                   ('pca', PCA(n_components=int(
                                       math.log2(len(X_train.columns))))),
                                   ('rf_reg', RandomForestRegressor())])]

grid_params_transformed_rf = [{'rf_reg__n_estimators': range(138, 143, 2),
                               'rf_reg__max_features': ['log2']}]

best_models_transformed_rf = []
for model in zip(rf_models, grid_params_transformed_rf):
    gs = GridSearchCV(estimator=model[0],
                      param_grid=model[1],
                      scoring='neg_mean_squared_error',
                      cv=5)
    if type(y_train) != np.ndarray:
        y = y_train.values.ravel()
        y_train = np.array(y).astype(float)
    gs.fit(X_train, y_train.ravel())
    best_models_transformed_rf.append(
        (sqrt(-1 * gs.best_score_), gs.best_params_, model[0]))

print(best_models_transformed_rf)

[(0.469774117760618, {'rf_reg__max_features': 'log2', 'rf_reg__n_estimators': 138}, Pipeline(memory=None,
     steps=[('rf_reg', RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False))]))]


In [164]:
best_transformed_rf_model = sorted(
    best_models_transformed_rf, key=lambda model: model[0])[0]
best_transformed_rf_model[1]['max_features'] = best_transformed_rf_model[1].pop(
    'rf_reg__max_features')
best_transformed_rf_model[1]['n_estimators'] = best_transformed_rf_model[1].pop(
    'rf_reg__n_estimators')
best_transformed_rf_model = (
    best_transformed_rf_model[0], best_transformed_rf_model[1], best_transformed_rf_model[2])

In [166]:
best_transformed_rf_model = sorted(
    best_models_transformed_rf, key=lambda model: model[0])[0]
best_transformed_rf_model = best_transformed_rf_model[2].steps[0][1].__class__(
    **best_transformed_rf_model[1])
best_transformed_rf_model.fit(train_data, train_target)
transformed_rf_predicted = best_transformed_rf_model.predict(test_data)
print(transformed_rf_predicted[:5])
print(test_target[:5])

/Users/robertsandor/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """


[2.23543478 2.20797101 2.29949275 2.23333333 2.16615942]
       relevance
49710       2.67
58951       1.33
45702       2.33
63338       2.00
71218       2.00


## Testing Smaller Feature Space

We decided to also test out how using only certain 'categories' of our features would affect our model. We grouped our model into similarity features (Jaro index, Jaccard index, etc.),  count features, and length/entropy features. 

We then tested out the best models using only those subsets of features to compare to the performance of the model with the full set of features. This is different than simply using PCA or other dimensionality reduction techniques because PCA may choose a few select features from each of the categories whereas this tests only features from one particular category.

In [17]:
all_num_features = modified_train[getAllNumericalCols(modified_train)]
all_num_features.head(3)

,stemmed_terms,clean_terms_in_title,clean_terms_in_desc,stemmed_terms_in_title,stemmed_terms_in_desc,lemmatized_terms_in_title,lemmatized_terms_in_desc,neighbours_in_title,neighbours_in_desc,search_terms_entropy,...,jscore_query_desc,jscore_query_title,search_title_SW,search_desc_SW,NCD_query_title,num_words_in_description,num_stop_words,num_search_words,tfidf_search_common,num_attrib
0,angl bracket,1.0,0.0,1.0,1.0,1.0,1.0,0.0,1.0,3.251629,...,2.833333,0.866667,1.0,4.0,0.107077,79,0,2,1,15.0
1,bracket,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.807355,...,0.000000,0.000000,0.0,0.0,0.107077,79,0,2,0,15.0
2,deck,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,2.000000,...,2.711111,0.000000,0.0,3.0,0.109091,109,1,2,1,35.0


In [25]:
all_similarity_features = modified_train[getSimilarityCols(all_num_features)]
all_similarity_features.head(3)

,neighbours_in_title,neighbours_in_desc,jaccard_index_title,jaccard_index_desc,lcs_title,lcs_desc,jscore_query_desc,jscore_query_title,search_title_SW,search_desc_SW,NCD_query_title
0,0.0,1.0,0.166667,0.000000,6,13,2.833333,0.866667,1.0,4.0,0.107077
1,0.0,0.0,0.000000,0.000000,3,7,0.000000,0.000000,0.0,0.0,0.107077
2,0.0,1.0,0.000000,0.012048,4,4,2.711111,0.000000,0.0,3.0,0.109091


In [26]:
count_cols, len_h_cols = getCountAndOtherCols(
    all_similarity_features, all_num_features)
all_count_features = modified_train[count_cols]
len_entropy_features = modified_train[len_h_cols]

In [27]:
train_data_numerical_subset, test_data_numerical_subset, train_target_numerical_subset, test_target_numerical_subset = train_test_split(all_num_features,
                                                                                                                                        y_train)

In [262]:
best_transformed_rf_model.fit(
    train_data_numerical_subset, train_target_numerical_subset)
transformed_rf_predicted_numerical_subset = best_transformed_rf_model.predict(
    test_data_numerical_subset)
print(transformed_rf_predicted_numerical_subset[:5])
print(test_target[:5])

/Users/robertsandor/anaconda3/envs/ml/lib/python3.6/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


[2.24535714 2.49071429 2.64038095 2.60203571 2.58490476]
       relevance
64115       2.33
55409       2.33
57088       2.00
38621       3.00
50452       2.33


In [266]:
best_rf_model.fit(train_data_numerical_subset, train_target_numerical_subset)
rf_predicted_numerical_subset = best_rf_model.predict(
    test_data_numerical_subset)
print(rf_predicted_numerical_subset[:5])
print(test_target[:5])

/Users/robertsandor/anaconda3/envs/ml/lib/python3.6/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


[2.22330935 2.45896043 2.65258993 2.58966427 2.46791367]
       relevance
64115       2.33
55409       2.33
57088       2.00
38621       3.00
50452       2.33


In [263]:
train_data_similarity_subset, test_data_similarity_subset, train_target_similarity_subset, test_target_similarity_subset = train_test_split(all_similarity_features,
                                                                                                                                            y_train)

In [264]:
best_transformed_rf_model.fit(
    train_data_similarity_subset, train_target_similarity_subset)
transformed_rf_predicted_similarity_subset = best_transformed_rf_model.predict(
    test_data_similarity_subset)
print(transformed_rf_predicted_similarity_subset[:5])
print(test_target[:5])

/Users/robertsandor/anaconda3/envs/ml/lib/python3.6/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


[2.11857143 2.27058673 2.1015     2.44828571 2.39452381]
       relevance
64115       2.33
55409       2.33
57088       2.00
38621       3.00
50452       2.33


In [268]:
best_rf_model.fit(train_data_similarity_subset, train_target_similarity_subset)
rf_predicted_similarity_subset = best_rf_model.predict(
    test_data_similarity_subset)
print(rf_predicted_similarity_subset[:5])
print(test_target[:5])

/Users/robertsandor/anaconda3/envs/ml/lib/python3.6/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


[2.02188849 2.34651079 2.10170264 2.58411871 2.42079137]
       relevance
64115       2.33
55409       2.33
57088       2.00
38621       3.00
50452       2.33


In [273]:
train_data_count_subset, test_data_count_subset, train_target_count_subset, test_target_count_subset = train_test_split(all_count_features,
                                                                                                                        y_train)

In [274]:
best_transformed_rf_model.fit(
    train_data_count_subset, train_target_count_subset)
transformed_rf_predicted_count_subset = best_transformed_rf_model.predict(
    test_data_count_subset)
print(transformed_rf_predicted_count_subset[:5])
print(test_target[:5])

/Users/robertsandor/anaconda3/envs/ml/lib/python3.6/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


[2.62299464 2.61285714 2.37335278 2.22497168 2.24638285]
       relevance
64115       2.33
55409       2.33
57088       2.00
38621       3.00
50452       2.33


In [275]:
best_rf_model.fit(train_data_count_subset, train_target_count_subset)
rf_predicted_count_subset = best_rf_model.predict(test_data_count_subset)
print(rf_predicted_count_subset[:5])
print(test_target[:5])

/Users/robertsandor/anaconda3/envs/ml/lib/python3.6/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


[2.62617557 2.52879496 2.36541988 2.20950156 2.27060029]
       relevance
64115       2.33
55409       2.33
57088       2.00
38621       3.00
50452       2.33


In [278]:
train_data_len_entropy_subset, test_data_len_entropy_subset, train_target_len_entropy_subset, test_target_len_entropy_subset = train_test_split(len_entropy_features,
                                                                                                                                                y_train)

In [279]:
best_transformed_rf_model.fit(
    train_data_len_entropy_subset, train_target_len_entropy_subset)
transformed_rf_predicted_len_entropy_subset = best_transformed_rf_model.predict(
    test_data_len_entropy_subset)
print(transformed_rf_predicted_len_entropy_subset[:5])
print(test_target[:5])

/Users/robertsandor/anaconda3/envs/ml/lib/python3.6/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


[2.54094286 2.59914286 2.36814286 2.59955051 2.06421429]
       relevance
64115       2.33
55409       2.33
57088       2.00
38621       3.00
50452       2.33


In [280]:
best_rf_model.fit(train_data_len_entropy_subset,
                  train_target_len_entropy_subset)
rf_predicted_len_entropy_subset = best_rf_model.predict(
    test_data_len_entropy_subset)
print(rf_predicted_len_entropy_subset[:5])
print(test_target[:5])

/Users/robertsandor/anaconda3/envs/ml/lib/python3.6/site-packages/ipykernel_launcher.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """Entry point for launching an IPython kernel.


[2.51521223 2.61115108 2.4346283  2.65931655 1.94434053]
       relevance
64115       2.33
55409       2.33
57088       2.00
38621       3.00
50452       2.33


## Evaluation Metric

The benchmark was ~rank 1681 on the Kaggle leaderboard for this competition with an RMSE of .51049

1st place had an RMSE of .43192

https://www.kaggle.com/c/home-depot-product-search-relevance/leaderboard

In [77]:
rmse_lin_reg = sqrt(mean_squared_error(predicted, test_target))

print(f"{rmse_lin_reg:.4f}")

0.4898


In [78]:
rmse_lin_reg_scaled = sqrt(mean_squared_error(scaled_linear_predicted, test_target))

print(f"{rmse_lin_reg_scaled:.4f}")

0.4981


In [224]:
rmse_tree = sqrt(mean_squared_error(tree_predicted, test_target))

print(f"{rmse_tree:.4f}")

0.4758


In [225]:
rmse_alt_linear = sqrt(mean_squared_error(alt_linear_predicted, test_target))

print(f"{rmse_alt_linear:.4f}")

0.4943


In [226]:
rmse_rf = sqrt(mean_squared_error(rf_predicted, test_target))

print(f"{rmse_rf:.4f}")

0.4688


In [227]:
rmse_transformed_rf = sqrt(mean_squared_error(
    transformed_rf_predicted, test_target))

print(f"{rmse_transformed_rf:.4f}")

0.4683


In [79]:
rmse_gb = sqrt(mean_squared_error(
    gb_predicted, test_target))

print(f"{rmse_gb:.4f}")

0.4786


### Reduced Subset RMSE - Random Forest after PCA

In [240]:
rmse_transformed_rf_numerical_subset = sqrt(mean_squared_error(
    transformed_rf_predicted_numerical_subset, test_target))

print(f"{rmse_transformed_rf_numerical_subset:.4f}")

0.5992


In [265]:
rmse_transformed_rf_predicted_similarity_subset = sqrt(
    mean_squared_error(transformed_rf_predicted_similarity_subset, test_target))

print(f"{transformed_rf_predicted_similarity_subset:.4f}")

0.5953


In [276]:
rmse_transformed_rf_count_subset = sqrt(mean_squared_error(
    transformed_rf_predicted_count_subset, test_target))

print(f"{rmse_transformed_rf_count_subset:.4f}")

0.6039


In [281]:
rmse_transformed_rf_len_entropy_subset = sqrt(mean_squared_error(
    transformed_rf_predicted_len_entropy_subset, test_target))

print(f"{rmse_transformed_rf_len_entropy_subset:.4f}")

0.5788


### Reduced Subset RMSE - Regular Random Forest

In [267]:
rmse_rf_numerical_subset = sqrt(mean_squared_error(
    rf_predicted_numerical_subset, test_target))

print(f"{rmse_rf_numerical_subset:.4f}")

0.5988


In [269]:
rmse_rf_similarity_subset = sqrt(mean_squared_error(
    rf_predicted_similarity_subset, test_target))

print(f"{rmse_rf_similarity_subset:.4f}")

0.5950


In [277]:
rmse_rf_count_subset = sqrt(mean_squared_error(
    rf_predicted_count_subset, test_target))

print(f"{rmse_rf_count_subset:.4f}")

0.6036


In [282]:
rmse_rf_len_entropy_subset = sqrt(mean_squared_error(
    rf_predicted_len_entropy_subset, test_target))

print(f"{rmse_rf_len_entropy_subset:.4f}")

0.5786
